In [1]:
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import wordnet as wn
import torch.nn.functional as F
import os
import glob
import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import pytorch_lightning as pl
import wandb
import re
#%load_ext tensorboard

/mnt/sw/nix/store/pvz51gzx2lyqpabf08xhd1y2kvw5rfdy-python-3.9.16-view/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/sw/nix/store/d9jbiy4kghz72ak7mija92asihk8xcja-py-torchvision-0.15.2/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from model import AutoEncoder, HookedModel, DictionnaryLearner, default_hookedmodel, default_ae

In [7]:
def load_model(generic_path, wandb_id, epoch = -1):
    path = os.path.join(generic_path, wandb_id, 'checkpoints')
    if epoch == -1:
        ## Create a list of all the epochs with a regex
        epochs = [int(re.findall(r'epoch=(\d+)', x)[0]) for x in os.listdir(path)]
        ## Get the last epoch
        epoch = max(epochs)
    ## Get the ckpt file starting with epoch={epoch}
    ckpt = [x for x in os.listdir(path) if x.startswith(f'epoch={epoch}')][0]
    model = DictionnaryLearner.load_from_checkpoint(os.path.join(path, ckpt))

In [18]:
#wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: david-heurtel-depeiges. Use `wandb login --relogin` to force relogin


True

In [24]:
## Config

## Data
TRAIN_PATH = '/tmp/imagenet/train'
VAL_PATH = '/tmp/imagenet/valid'

## Data transforms
PATCH_SIZE = 'Full'

## Model 
D_HIDDEN = 1024
CHANNELS = 508
D_MULT = 1 ## If not 1, D_HIDDEN = CHANNELS * D_MULT
if D_MULT != 1:
    D_HIDDEN = CHANNELS * D_MULT
L1_COEFF = 3e-4


MODEL_NAME = 'inceptionv1_mixed4a_monosemantic'

## Training
BATCH_SIZE = 4096
OPTIM = 'Adam'


## Checkpointing
SAVE_PATH = '/path/to/checkpoints/monosemantic/'

## Logging

WANBD_PROJECT = 'monosemantic_dictionnary_learning'

os.makedirs(SAVE_PATH, exist_ok=True)

In [6]:
CONFIG = {
    'dataset': 'imagenet',
    'patch_size': PATCH_SIZE,
    'd_input' : CHANNELS,
    'd_hidden': D_HIDDEN,
    'l1_coeff': L1_COEFF,
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'optim': OPTIM,
    'save_path': SAVE_PATH,
    'wandb_project': WANBD_PROJECT, 
}

In [ ]:
dirnotempy = os.path.exists('/tmp/imagenet/train')
if dirnotempy:
    l = os.listdir('/tmp/imagenet/train')
    if len(l)==0:
        dirnotempy = False
if not dirnotempy:
    os.system('bash dataload.sh')

traindir = os.path.join('/tmp/imagenet/', 'train')
valdir = os.path.join('/tmp/imagenet/', 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
def unormalize(batch):
    ## batch of shape (batch_size, 3, 224, 224)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)

    return batch * std + mean

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

val_dataset = datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))    

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = load_model('/path/to/checkpoints/monosemantic/inceptionv1_mixed4a_monosemantic/monosemantic_dictionnary_learning/', 'w6oqqydk', -1)

In [9]:
dict_learner = DictionnaryLearner(d_hidden = 10 * 508, d_input = 508, l1_coeff = 1e-2, layer_to_hook='mixed5b', patch_size = None).to(device)

/mnt/home/dheurtel/venv/genv_DL/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'layer_to_hook' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['layer_to_hook'])`.


In [10]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=8)

NameError: name 'train_dataset' is not defined

In [26]:
logger_type = 'tensorboard'

In [27]:
if logger_type.lower()=='wandb':
    logger = pl.loggers.WandbLogger(project=WANBD_PROJECT, config=CONFIG)
    logger.watch(dict_learner)
    logger.log_hyperparams(CONFIG)
    default_root_dir = None
elif logger_type.lower()=='tensorboard':
    logger = pl.loggers.TensorBoardLogger(SAVE_PATH, name=MODEL_NAME)
    default_root_dir = None
else:
    logger = None
    default_root_dir = SAVE_PATH

logger = pl.loggers.TensorBoardLogger(SAVE_PATH, name=MODEL_NAME)

trainer = pl.Trainer(gpus=1, max_epochs=1, progress_bar_refresh_rate=20,default_root_dir=default_root_dir, logger=logger)
trainer.fit(dict_learner, train_dataloaders=dummy_dataloader, val_dataloaders=dummy_dataloader)

/mnt/sw/nix/store/pvz51gzx2lyqpabf08xhd1y2kvw5rfdy-python-3.9.16-view/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type        | Params
--------------------------------------------
0 | autoencoder | AutoEncoder | 5.2 M 
1 | hookedmodel | HookedModel | 7.1 M 
--------------------------------------------
5.2 M     Trainable params
7.1 M     Non-trainable params
12.2 M    Total params
48.933    Total esti

Validation sanity check: 0it [00:00, ?it/s]

/mnt/sw/nix/store/pvz51gzx2lyqpabf08xhd1y2kvw5rfdy-python-3.9.16-view/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/mnt/sw/nix/store/pvz51gzx2lyqpabf08xhd1y2kvw5rfdy-python-3.9.16-view/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:407: UserWarning: The number of training samples (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]